In [25]:
import sys
!{sys.executable} -m pip install xgboost

You should consider upgrading via the '/Users/jeankeslernunes/Documents/FIAP/ClickPlus_Challenge_Sprint_4/.venv/bin/python -m pip install --upgrade pip' command.


Bloco 1: Carregamento dos Dados


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Carregando o DataFrame RFM para a modelagem
caminho_rfm = '../data/features/rfm_features.parquet'
df_rfm = pd.read_parquet(caminho_rfm)

# Carregando o DataFrame de Vendas para criar o dataset de treino da predição
caminho_sales = '../data/redis/sales.parquet'
df_sales = pd.read_parquet(caminho_sales)

# Carregando o DataFrame 'customers' que será atualizado no final
caminho_customers = '../data/redis/customers.parquet'
df_customers = pd.read_parquet(caminho_customers)

print("Todos os dados foram carregados com sucesso!")

Todos os dados foram carregados com sucesso!


Bloco 2: Definição do Período de Treino e Teste (Corte Temporal)


In [27]:
# Encontrando a última data de compra no dataset
ultima_data = df_sales['data_venda'].max()
print(f"A última compra registrada foi em: {ultima_data.date()}")

#  Definindo nossa data de corte 30 dias antes do final
data_corte = ultima_data - pd.Timedelta(days=30)
print(f"Nossa data de corte para a análise será: {data_corte.date()}")

# Separando os dados em treino (passado) e teste (futuro)
df_treino = df_sales[df_sales['data_venda'] <= data_corte]
df_futuro = df_sales[df_sales['data_venda'] > data_corte]

print(f"\nTemos {len(df_treino)} registros de treino (passado) e {len(df_futuro)} registros no 'futuro' (gabarito).")

A última compra registrada foi em: 2024-04-01
Nossa data de corte para a análise será: 2024-03-02

Temos 98214 registros de treino (passado) e 1786 registros no 'futuro' (gabarito).


Bloco 3: Criar os "Gabaritos" de compras em 7 dias e 30 dias

In [28]:
# 1. Gabarito para 30 dias
clientes_que_compraram_30d = df_futuro['id_cliente'].unique()
print(f"Encontramos {len(clientes_que_compraram_30d)} clientes únicos que compraram nos últimos 30 dias.")

# 2. Gabarito para 7 dias
data_corte_7d = data_corte + pd.Timedelta(days=7)
df_futuro_7d = df_sales[(df_sales['data_venda'] > data_corte) & (df_sales['data_venda'] <= data_corte_7d)]
clientes_que_compraram_7d = df_futuro_7d['id_cliente'].unique()
print(f"Encontramos {len(clientes_que_compraram_7d)} clientes únicos que compraram nos primeiros 7 dias do período futuro.")

Encontramos 1543 clientes únicos que compraram nos últimos 30 dias.
Encontramos 346 clientes únicos que compraram nos primeiros 7 dias do período futuro.


Bloco 4: Criar as Features (RFM) Usando Apenas os Dados de Treino


In [29]:
# Calculando o RFM para todos os clientes, mas usando apenas o histórico de treino
print("Calculando as features RFM com base nos dados de treino...")

# A data de referência agora é a nossa data de corte
snapshot_date_treino = data_corte + pd.Timedelta(days=1)

# Agrupando por cliente e calculando RFM usando apenas o df_treino
df_rfm_treino = df_treino.groupby('id_cliente').agg({
    'data_venda': lambda date: (snapshot_date_treino - date.max()).days, # Recência
    'id_cliente': 'count', # Frequência (simplificada para o modelo)
    'valor_venda': 'sum' # Valor Monetário
})

# Renomeando as colunas
df_rfm_treino.rename(columns={'data_venda': 'Recencia',
                              'id_cliente': 'Frequencia',
                              'valor_venda': 'ValorMonetario'}, inplace=True)

print("Features RFM calculadas para o período de treino:")
display(df_rfm_treino.head())

Calculando as features RFM com base nos dados de treino...
Features RFM calculadas para o período de treino:


,Recencia,Frequencia,ValorMonetario
id_cliente,,,
0000029b76ad3cf9d86ad430754fb1d4478069affda61e8adaf4c57e9aa4b37b,1149,1,91.02
000010ae2e13049769982d9f07de792d92452ff1d124e3a49109fa57f6af54b8,668,1,82.48
0001018716456b2b34ca7a31f9b597974be6e1c9f6122a1bba5bb9c267a9e7fe,229,1,193.61
0002fdedbce706df9a2602a5e1286a904e4582264572e10b0988c607e84288f7,782,1,385.83
0003cb164c9b94826ccad40eaa46efc7bd2242d6c4625370a6609fcc1a263da4,66,1,116.22


Bloco 5: Criar a Variável-Alvo e Juntar Tudo


In [30]:
# Criando as duas variáveis-alvo (target)
df_rfm_treino['comprou_em_30d'] = df_rfm_treino.index.isin(clientes_que_compraram_30d).astype(int)
df_rfm_treino['comprou_em_7d'] = df_rfm_treino.index.isin(clientes_que_compraram_7d).astype(int)

# Criando nosso dataset final para o modelo
df_modelo = df_rfm_treino.copy()

print("\nDataset final para o modelo, com features e variáveis-alvo:")
display(df_modelo.head())

# Verificando quantos clientes compraram (1) vs. não compraram (0) para cada alvo
print("\nDistribuição da variável-alvo para 30 dias:")
print(df_modelo['comprou_em_30d'].value_counts())

print("\nDistribuição da variável-alvo para 7 dias:")
print(df_modelo['comprou_em_7d'].value_counts())


Dataset final para o modelo, com features e variáveis-alvo:


,Recencia,Frequencia,ValorMonetario,comprou_em_30d,comprou_em_7d
id_cliente,,,,,
0000029b76ad3cf9d86ad430754fb1d4478069affda61e8adaf4c57e9aa4b37b,1149,1,91.02,0,0
000010ae2e13049769982d9f07de792d92452ff1d124e3a49109fa57f6af54b8,668,1,82.48,0,0
0001018716456b2b34ca7a31f9b597974be6e1c9f6122a1bba5bb9c267a9e7fe,229,1,193.61,0,0
0002fdedbce706df9a2602a5e1286a904e4582264572e10b0988c607e84288f7,782,1,385.83,0,0
0003cb164c9b94826ccad40eaa46efc7bd2242d6c4625370a6609fcc1a263da4,66,1,116.22,0,0



Distribuição da variável-alvo para 30 dias:
comprou_em_30d
0    53138
1      706
Name: count, dtype: int64

Distribuição da variável-alvo para 7 dias:
comprou_em_7d
0    53665
1      179
Name: count, dtype: int64


# Bloco 6: Treinando e Avaliando o Modelo Preditivo


In [31]:
# Bloco 6 - NOVO E COMPLETO

# 1. Importando as bibliotecas necessárias
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score

# 2. Preparando os dados para os modelos
# As features de entrada (X) são as mesmas para ambos os modelos
X = df_rfm_treino[['Recencia', 'Frequencia', 'ValorMonetario']]

# Nossas duas variáveis-alvo (y) distintas
y_30d = df_rfm_treino['comprou_em_30d']
y_7d = df_rfm_treino['comprou_em_7d']


# --- TREINO E AVALIAÇÃO DO MODELO DE 30 DIAS ---
print("--- INICIANDO TREINO DO MODELO DE 30 DIAS ---")

# 3.1. Dividindo os dados para o alvo de 30 dias
# 'stratify' garante que a proporção de compradores e não compradores seja a mesma no treino e no teste.
X_train_30d, X_test_30d, y_train_30d, y_test_30d = train_test_split(
    X, y_30d, test_size=0.2, random_state=42, stratify=y_30d
)

# 3.2. Criando e Treinando o Modelo XGBoost para 30 dias
# 'scale_pos_weight' é a forma mais eficaz de lidar com dados desbalanceados no XGBoost.
weight_30d = sum(y_train_30d == 0) / sum(y_train_30d == 1)
model_30d = xgb.XGBClassifier(random_state=42, scale_pos_weight=weight_30d, use_label_encoder=False, eval_metric='logloss')

print("Treinando o modelo de 30 dias...")
model_30d.fit(X_train_30d, y_train_30d)
print("Modelo de 30 dias treinado com sucesso!")

# 3.3. Fazendo as previsões no conjunto de teste de 30 dias
y_pred_30d = model_30d.predict(X_test_30d)
y_prob_30d = model_30d.predict_proba(X_test_30d)[:, 1]

# 3.4. Avaliando a performance do modelo de 30 dias
print("\n--- Relatório de Classificação (30 dias) ---")
print(classification_report(y_test_30d, y_pred_30d))
auc_30d = roc_auc_score(y_test_30d, y_prob_30d)
print(f"AUC-ROC Score (30 dias): {auc_30d:.4f}")


# --- TREINO E AVALIAÇÃO DO MODELO DE 7 DIAS ---
print("\n\n--- INICIANDO TREINO DO MODELO DE 7 DIAS ---")

# 4.1. Dividindo os dados para o alvo de 7 dias
X_train_7d, X_test_7d, y_train_7d, y_test_7d = train_test_split(
    X, y_7d, test_size=0.2, random_state=42, stratify=y_7d
)

# 4.2. Criando e Treinando o Modelo XGBoost para 7 dias
weight_7d = sum(y_train_7d == 0) / sum(y_train_7d == 1)
model_7d = xgb.XGBClassifier(random_state=42, scale_pos_weight=weight_7d, use_label_encoder=False, eval_metric='logloss')

print("Treinando o modelo de 7 dias...")
model_7d.fit(X_train_7d, y_train_7d)
print("Modelo de 7 dias treinado com sucesso!")

# 4.3. Fazendo as previsões no conjunto de teste de 7 dias
y_pred_7d = model_7d.predict(X_test_7d)
y_prob_7d = model_7d.predict_proba(X_test_7d)[:, 1]

# 4.4. Avaliando a performance do modelo de 7 dias
print("\n--- Relatório de Classificação (7 dias) ---")
print(classification_report(y_test_7d, y_pred_7d))
auc_7d = roc_auc_score(y_test_7d, y_prob_7d)
print(f"AUC-ROC Score (7 dias): {auc_7d:.4f}")

--- INICIANDO TREINO DO MODELO DE 30 DIAS ---
Treinando o modelo de 30 dias...


/Users/jeankeslernunes/Documents/FIAP/ClickPlus_Challenge_Sprint_4/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:58:09] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Modelo de 30 dias treinado com sucesso!

--- Relatório de Classificação (30 dias) ---
              precision    recall  f1-score   support

           0       0.99      0.93      0.96     10628
           1       0.06      0.33      0.10       141

    accuracy                           0.92     10769
   macro avg       0.52      0.63      0.53     10769
weighted avg       0.98      0.92      0.95     10769

AUC-ROC Score (30 dias): 0.7265


--- INICIANDO TREINO DO MODELO DE 7 DIAS ---
Treinando o modelo de 7 dias...


/Users/jeankeslernunes/Documents/FIAP/ClickPlus_Challenge_Sprint_4/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:58:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Modelo de 7 dias treinado com sucesso!

--- Relatório de Classificação (7 dias) ---
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     10733
           1       0.05      0.17      0.07        36

    accuracy                           0.99     10769
   macro avg       0.52      0.58      0.53     10769
weighted avg       0.99      0.99      0.99     10769

AUC-ROC Score (7 dias): 0.7851


Nosso primeiro modelo é cauteloso e conservador. Ele prefere não arriscar um palpite errado (por isso o Recall é baixo, ele perde muitas oportunidades), mas dos poucos palpites que ele dá, uma porção razoável está correta (Precisão de 36%). A nota AUC de 0.77 nos confirma que as features RFM que criamos são sim preditivas e que estamos no caminho certo. Na próxima fase, pretendemos melhorar as features, testar outros modelos, como XGBoost, e ajustar o ponto de corte para otimizar a precisão e/ou o recall.

Bloco Final: Gerar predições para TODOS os clientes e atualizar o arquivo final


In [32]:
# --- Bloco Final: Gerar Predições e Atualizar Arquivo de Saída ---

print("Iniciando a etapa final de predição e salvamento...")

# --- ETAPA 1: PREPARAR OS DADOS FINAIS ---

# Usaremos todo o histórico de treino (df_rfm_treino) para treinar os modelos finais
X_final_treino = df_rfm_treino[['Recencia', 'Frequencia', 'ValorMonetario']]
y_final_treino_30d = df_rfm_treino['comprou_em_30d']
y_final_treino_7d = df_rfm_treino['comprou_em_7d']

# E usaremos a base RFM completa (df_rfm), que representa todos os clientes, para gerar as previsões
X_para_prever = df_rfm[['Recencia', 'Frequencia', 'ValorMonetario']]


# --- ETAPA 2: TREINAR OS MODELOS FINAIS ---
# Esta é a parte que estava faltando. Treinamos os modelos com 100% dos dados históricos.

print("\n--- Treinando o MODELO FINAL de 30 dias com todos os dados históricos...")
weight_final_30d = sum(y_final_treino_30d == 0) / sum(y_final_treino_30d == 1)
model_final_30d = xgb.XGBClassifier(random_state=42, scale_pos_weight=weight_final_30d, use_label_encoder=False, eval_metric='logloss')
model_final_30d.fit(X_final_treino, y_final_treino_30d)
print("Modelo final de 30 dias treinado com sucesso!")

print("\n--- Treinando o MODELO FINAL de 7 dias com todos os dados históricos...")
weight_final_7d = sum(y_final_treino_7d == 0) / sum(y_final_treino_7d == 1)
model_final_7d = xgb.XGBClassifier(random_state=42, scale_pos_weight=weight_final_7d, use_label_encoder=False, eval_metric='logloss')
model_final_7d.fit(X_final_treino, y_final_treino_7d)
print("Modelo final de 7 dias treinado com sucesso!")


# --- ETAPA 3: GERAR PREVISÕES FINAIS E ATUALIZAR O ARQUIVO ---
print("\n--- Gerando probabilidades reais para todos os clientes...")

# Agora as variáveis 'model_final_30d' e 'model_final_7d' existem e podemos usá-las
probabilidades_30d = model_final_30d.predict_proba(X_para_prever)[:, 1]
probabilidades_7d = model_final_7d.predict_proba(X_para_prever)[:, 1]

df_rfm['prob_compra_30d'] = probabilidades_30d.round(2)
df_rfm['prob_compra_7d'] = probabilidades_7d.round(2)

df_rfm_com_id = df_rfm.reset_index().rename(columns={'fk_contact': 'id_cliente'})

df_customers_para_merge = df_customers.drop(
    columns=['prob_compra_7d', 'prob_compra_30d', 'prob_prox_compra'], 
    errors='ignore'
)

df_customers_atualizado = pd.merge(
    df_customers_para_merge,
    df_rfm_com_id[['id_cliente', 'prob_compra_7d', 'prob_compra_30d']],
    on='id_cliente',
    how='left'
)

colunas_finais = [
    'id_cliente', 
    'nome_cliente',
    'segmento',
    'prob_compra_7d',
    'prob_compra_30d', 
    'sugestao_prox_trecho', 
    'data_prox_viagem'
]
df_customers_atualizado = df_customers_atualizado[colunas_finais]
df_customers_atualizado.to_parquet(caminho_customers, index=False)

print(f"\nArquivo '{caminho_customers}' atualizado com sucesso com as probabilidades REAIS dos modelos de 7 e 30 dias!")
display(df_customers_atualizado.head())


Iniciando a etapa final de predição e salvamento...

--- Treinando o MODELO FINAL de 30 dias com todos os dados históricos...


/Users/jeankeslernunes/Documents/FIAP/ClickPlus_Challenge_Sprint_4/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:58:10] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Modelo final de 30 dias treinado com sucesso!

--- Treinando o MODELO FINAL de 7 dias com todos os dados históricos...


/Users/jeankeslernunes/Documents/FIAP/ClickPlus_Challenge_Sprint_4/.venv/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [22:58:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Modelo final de 7 dias treinado com sucesso!

--- Gerando probabilidades reais para todos os clientes...

Arquivo '../data/04_output/customers.parquet' atualizado com sucesso com as probabilidades REAIS dos modelos de 7 e 30 dias!


,id_cliente,nome_cliente,segmento,prob_compra_7d,prob_compra_30d,sugestao_prox_trecho,data_prox_viagem
0,0000029b76ad3cf9d86ad430754fb1d4478069affda61e...,Dra. Raquel Duarte,Em Risco,0.00,0.01,Aguardando recomendação,Aguardando previsão
1,000010ae2e13049769982d9f07de792d92452ff1d124e3...,Léo Silveira,Em Risco,0.00,0.13,Aguardando recomendação,Aguardando previsão
2,0001018716456b2b34ca7a31f9b597974be6e1c9f6122a...,Mariah Caldeira,Em Risco,0.05,0.26,Aguardando recomendação,Aguardando previsão
3,0002fdedbce706df9a2602a5e1286a904e4582264572e1...,Liz Pinto,Em Risco,0.00,0.04,Aguardando recomendação,Aguardando previsão
4,0003cb164c9b94826ccad40eaa46efc7bd2242d6c46253...,Maria Julia Jesus,Em Risco,0.00,0.30,Aguardando recomendação,Aguardando previsão
